In [15]:
import pandas as pd
import os
from datetime import datetime

class DataTransformer:
    def __init__(self, stock_codes, table_names, input_folder="parsed_data_raw", output_folder="processed_data"):
        self.stock_codes = stock_codes
        self.table_names = table_names
        self.input_folder = input_folder
        self.output_folder = output_folder

    def process_financial_report(self, stock_code, table_name):
        input_folder_path = os.path.join("..", "..", "parsed_data_raw", "csv", table_name, f"{stock_code}_csv")
        output_folder_path = os.path.join(self.output_folder, "csv", table_name, f"{stock_code}_processed")
        os.makedirs(output_folder_path, exist_ok=True)

        all_results = []  # Danh sách lưu tất cả kết quả

        for file_name in os.listdir(input_folder_path):
            if file_name.endswith(".csv"):
                input_csv = os.path.join(input_folder_path, file_name)
                print(f"🔄 Đang xử lý: {input_csv}")

                df = pd.read_csv(input_csv, header=0).dropna(how='all')
                quarters = df.columns[1:]

                records = []
                for _, row in df.iterrows():
                    indicator = str(row.iloc[0]).strip()
                    if indicator in ["Giai đoạn", "Hợp nhất", "Kiểm toán", "Công ty kiểm toán", "Ý kiến kiểm toán"]:
                        continue

                    for quarter in quarters:
                        quarter_formatted = quarter.replace("/", "")
                        report_id = f"{stock_code}BCKQKD{quarter_formatted}"
                        value = str(row[quarter]).replace(",", "")

                        if value in ["", "nan", "None"]:
                            continue

                        ingestion_date = updated_date = datetime.now().strftime("%Y-%m-%d")
                        records.append([report_id, indicator, value, ingestion_date, updated_date])

                df_result = pd.DataFrame(records, columns=["report_id", "index", "value", "ingestion_date", "updated_date"])
                all_results.append(df_result)  # Thêm kết quả vào danh sách

                # Đặt tên file đầu ra dựa trên file gốc
                output_csv = os.path.join(output_folder_path, f"processed_{file_name}")
                df_result.to_csv(output_csv, index=False, encoding="utf-8")
                
                print(f"✅ File đã được lưu tại: {output_csv}")

        return all_results  # Trả về danh sách kết quả thay vì một file duy nhất

    def run(self, table_name):
        for stock_code in self.stock_codes:
            print(f"🔄 Đang xử lý: {stock_code} - {table_name}")
            self.process_financial_report(stock_code, table_name)

if __name__ == "__main__":
    stock_codes = [
        "ACB", "BCM", "BID", "BVH", "CTG", "FPT", "GAS", "GVR", "HDB", "HPG",
        "LPB", "MBB", "MSN", "MWG", "PLX", "SAB", "SHB", "SSB", "SSI", "STB",
        "TCB", "TPB", "VCB", "VHM", "VIC", "VJC", "VNM", "VPB", "VRE"
    ]

    table_names = {"inc_state", "balance", "cash_flow"}

    transformer = DataTransformer(stock_codes, table_names)
    transformer.run(table_name="balance")


🔄 Đang xử lý: ACB - balance
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_1.csv
✅ File đã được lưu tại: processed_data\csv\balance\ACB_processed\processed_ACB_tbl-data-CDKT_1.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_10.csv
✅ File đã được lưu tại: processed_data\csv\balance\ACB_processed\processed_ACB_tbl-data-CDKT_10.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_11.csv
✅ File đã được lưu tại: processed_data\csv\balance\ACB_processed\processed_ACB_tbl-data-CDKT_11.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_12.csv
✅ File đã được lưu tại: processed_data\csv\balance\ACB_processed\processed_ACB_tbl-data-CDKT_12.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_13.csv
✅ File đã được lưu tại: processed_data\csv\balance\ACB_processed\processed_ACB_tbl-data-CDKT_13.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-

In [8]:
import os

input_folder = "parsed_data_raw"
table_name = "balance"
stock_code = "ACB"

input_folder_path = os.path.join("..", "..", "parsed_data_raw", "csv", table_name, f"{stock_code}_csv")

if os.path.exists(input_folder_path):
    print(f"✅ Thư mục tồn tại: {input_folder_path}")
else:
    print(f"❌ Thư mục KHÔNG tồn tại: {input_folder_path}")


✅ Thư mục tồn tại: ..\..\parsed_data_raw\csv\balance\ACB_csv


In [7]:
input_folder_path = os.path.abspath(os.path.join("..", "..", "parsed_data_raw", "csv", table_name, f"{stock_code}_csv"))
print(f"📂 Đường dẫn thực tế: {input_folder_path}")


📂 Đường dẫn thực tế: d:\Project\stock-crawler\parsed_data_raw\csv\balance\ACB_csv


Test 2

In [51]:
import csv
import os
import openpyxl
from collections import defaultdict

class DataTransformer:
    def __init__(self, stock_codes, input_folder="parsed_data_raw", output_folder="processed_data"):
        self.stock_codes = stock_codes
        self.input_folder = input_folder
        self.output_folder = output_folder

    def process_financial_report(self, stock_code, table_name):
        """Xử lý báo cáo tài chính và giữ nguyên thứ tự index"""

        # 📁 Đường dẫn input và output
        input_folder_path = os.path.join("..", "..", "parsed_data_raw", "csv", table_name, f"{stock_code}_csv")
        output_folder_path = os.path.join(self.output_folder, "csv", table_name)
        os.makedirs(output_folder_path, exist_ok=True)

        data_dict = defaultdict(dict)  # Lưu dữ liệu dạng { (stock_code, time): {index: value} }
        index_order = []  # Giữ nguyên thứ tự xuất hiện của index

        # 📂 Đọc từng file CSV
        for file_name in os.listdir(input_folder_path):
            if file_name.endswith(".csv"):
                input_csv = os.path.join(input_folder_path, file_name)
                print(f"🔄 Đang xử lý: {input_csv}")

                with open(input_csv, "r", encoding="utf-8") as f:
                    reader = csv.reader(f)
                    headers = next(reader)  # Lấy tiêu đề

                    quarters = headers[1:]  # Các cột quý (VD: "2023/Q1", "2023/Q2")

                    for row in reader:
                        indicator = row[0].strip()

                        if indicator in ["Giai đoạn", "Hợp nhất", "Kiểm toán", "Công ty kiểm toán", "Ý kiến kiểm toán"]:
                            continue  # Bỏ qua các chỉ số không cần thiết

                        # 🔄 Duy trì thứ tự index
                        if indicator not in index_order:
                            index_order.append(indicator)

                        for i, quarter in enumerate(quarters):
                            quarter_formatted = quarter.replace("/", "")  # Chuẩn hóa định dạng thời gian
                            value = row[i + 1].replace(",", "").strip()

                            # ⚠️ Giữ nguyên giá trị rỗng
                            if value in ["", "nan", "None"]:
                                value = ""

                            # Chuyển đổi số nếu có thể
                            try:
                                value = float(value) if value else ""
                            except ValueError:
                                pass  # Nếu không phải số, giữ nguyên

                            data_dict[(stock_code, quarter_formatted)][indicator] = value

        # 📁 Ghi dữ liệu ra file CSV
        output_csv = os.path.join(output_folder_path, f"{stock_code}_transformed.csv")
        data_list = []

        with open(output_csv, "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)

            # 📝 Ghi header: ["stock_code", "time", chỉ số tài chính...]
            writer.writerow(["stock_code", "time"] + index_order)

            # 📝 Ghi dữ liệu theo thứ tự
            for (stock_code, time), values in sorted(data_dict.items()):
                row = [stock_code, time] + [values.get(index, "") for index in index_order]
                writer.writerow(row)
                data_list.append(row)

        print(f"✅ File CSV đã được lưu tại: {output_csv}")

        # 📁 Ghi dữ liệu ra file Excel
        output_excel_folder = os.path.join(self.output_folder, "excel", table_name)
        os.makedirs(output_excel_folder, exist_ok=True)  # Đảm bảo thư mục tồn tại
        output_excel = os.path.join(output_excel_folder, f"{stock_code}_processed.xlsx")
        
        self.save_to_excel(output_excel, ["stock_code", "time"] + index_order, data_list)

    def save_to_excel(self, file_path, headers, data):
        """Lưu dữ liệu vào file Excel"""
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.append(headers)  # Ghi tiêu đề

        for row in data:
            sheet.append(row)  # Ghi dữ liệu

        workbook.save(file_path)
        print(f"✅ File Excel đã được lưu tại: {file_path}")

    def run(self, table_name):
        for stock_code in self.stock_codes:
            print(f"\n🔄 Đang xử lý: {stock_code} - {table_name}")
            self.process_financial_report(stock_code, table_name)

if __name__ == "__main__":
    stock_codes = [
        "ACB", "BCM", "BID", "BVH", "CTG", "FPT", "GAS", "GVR", "HDB", "HPG",
        "LPB", "MBB", "MSN", "MWG", "PLX", "SAB", "SHB", "SSB",
        "SSI", "STB",
        "TCB", "TPB", "VCB", "VHM", "VIC", "VJC", "VNM", "VPB", "VRE"
    ]

    transformer = DataTransformer(stock_codes, table_names)
    transformer.run(table_name="balance")
    transformer.run(table_name="inc_state")
    transformer.run(table_name="cash_flow")





🔄 Đang xử lý: ACB - balance
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_1.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_10.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_11.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_12.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_13.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_14.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_15.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_2.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_3.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_4.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_5.csv
🔄 Đang xử lý: ..\..\parsed_data_raw\csv\balance\ACB_csv\ACB_tbl-data-CDKT_6.csv
🔄 Đan

In [ ]:
# import csv
# import os
# import openpyxl
# from collections import defaultdict

# class DataTransformer:
#     def __init__(self, stock_codes, table_names, input_folder="parsed_data_raw", output_folder="processed_data"):
#         self.stock_codes = stock_codes
#         self.table_names = table_names
#         self.input_folder = input_folder
#         self.output_folder = output_folder

#     def process_financial_report(self, stock_code, table_name):
#         """Xử lý báo cáo tài chính và giữ nguyên thứ tự index"""
        
#         # 📁 Đường dẫn input và output
#         input_folder_path = os.path.join("..", "..", "parsed_data_raw", "csv", table_name, f"{stock_code}_csv")
#         output_folder_path = os.path.join(self.output_folder, "csv", table_name, f"{stock_code}_processed")
#         os.makedirs(output_folder_path, exist_ok=True)

#         data_dict = defaultdict(dict)  # Lưu dữ liệu dạng { (stock_code, time): {index: value} }
#         index_order = []  # Giữ nguyên thứ tự xuất hiện của index

#         # 📂 Đọc từng file CSV
#         for file_name in os.listdir(input_folder_path):
#             if file_name.endswith(".csv"):
#                 input_csv = os.path.join(input_folder_path, file_name)
#                 print(f"🔄 Đang xử lý: {input_csv}")

#                 with open(input_csv, "r", encoding="utf-8") as f:
#                     reader = csv.reader(f)
#                     headers = next(reader)  # Lấy tiêu đề (bỏ qua)

#                     quarters = headers[1:]  # Các cột quý (VD: "2023/Q1", "2023/Q2")

#                     for row in reader:
#                         indicator = row[0].strip()

#                         if indicator in ["Giai đoạn", "Hợp nhất", "Kiểm toán", "Công ty kiểm toán", "Ý kiến kiểm toán"]:
#                             continue  # Bỏ qua các chỉ số không cần thiết

#                         # 🔄 Duy trì thứ tự index
#                         if indicator not in index_order:
#                             index_order.append(indicator)

#                         for i, quarter in enumerate(quarters):
#                             quarter_formatted = quarter.replace("/", "")  # Chuẩn hóa định dạng thời gian
#                             value = row[i + 1].replace(",", "").strip()

#                             if value not in ["", "nan", "None"]:  # Bỏ qua giá trị rỗng
#                                 data_dict[(stock_code, quarter_formatted)][indicator] = float(value)

#         # 📁 Ghi dữ liệu ra file CSV
#         output_csv = os.path.join(output_folder_path, f"{stock_code}_transformed.csv")
#         data_list = []

#         with open(output_csv, "w", encoding="utf-8", newline="") as f:
#             writer = csv.writer(f)

#             # 📝 Ghi header: ["stock_code", "time", chỉ số tài chính...]
#             writer.writerow(["stock_code", "time"] + index_order)

#             # 📝 Ghi dữ liệu theo thứ tự
#             for (stock_code, time), values in sorted(data_dict.items()):
#                 row = [stock_code, time] + [values.get(index, "") for index in index_order]
#                 writer.writerow(row)
#                 data_list.append(row)

#         print(f"✅ File CSV đã được lưu tại: {output_csv}")

#         # 📁 Ghi dữ liệu ra file Excel
#         output_excel = os.path.join(output_folder_path, f"{stock_code}_processed_excel.xlsx")
#         self.save_to_excel(output_excel, ["stock_code", "time"] + index_order, data_list)

#     def save_to_excel(self, file_path, headers, data):
#         """Lưu dữ liệu vào file Excel"""
#         workbook = openpyxl.Workbook()
#         sheet = workbook.active
#         sheet.append(headers)  # Ghi tiêu đề

#         for row in data:
#             sheet.append(row)  # Ghi dữ liệu

#         workbook.save(file_path)
#         print(f"✅ File Excel đã được lưu tại: {file_path}")

#     def run(self, table_name):
#         for stock_code in self.stock_codes:
#             print(f"\n🔄 Đang xử lý: {stock_code} - {table_name}")
#             self.process_financial_report(stock_code, table_name)

# if __name__ == "__main__":
#     stock_codes = [
#         "ACB", "BCM", "BID", "BVH", "CTG", "FPT", "GAS", "GVR", "HDB", "HPG",
#         "LPB", "MBB", "MSN", "MWG", "PLX", "SAB", "SHB", "SSB",
#         "SSI", "STB",
#         "TCB", "TPB", "VCB", "VHM", "VIC", "VJC", "VNM", "VPB", "VRE"
#     ]
#     table_names = {"balance"}

#     transformer = DataTransformer(stock_codes, table_names)
#     transformer.run(table_name="balance")


In [5]:
import os
import pandas as pd

groups_folder_path = os.path.join("processed_data/csv/balance")
group1 = []
group2 = []

# Lặp qua tất cả các file trong thư mục
for filename in os.listdir(groups_folder_path):
    if filename.endswith("_transformed.csv"):  # Chỉ xử lý các file có cấu trúc mong muốn
        file_path = os.path.join(groups_folder_path, filename)

        try:
            # Đọc chỉ tên cột của file CSV
            df = pd.read_csv(file_path, nrows=0)  # Đọc dòng header (tên cột)
            column_names = df.columns.astype(str).to_list()  # Chuyển danh sách cột thành string list

            stockcode = filename.replace("_transformed.csv", "")
            if any("Tiền và các khoản tương đương tiền" in col for col in column_names):
                group1.append(stockcode)
            else:
                group2.append(stockcode)

        except Exception as e:
            print(f"Lỗi khi đọc file {filename}: {e}")

print("Group 1:", group1)
print("Group 2:", group2)


Group 1: ['BCM', 'BVH', 'FPT', 'GAS', 'GVR', 'HPG', 'MSN', 'MWG', 'PLX', 'SAB', 'SSI', 'VHM', 'VIC', 'VJC', 'VNM', 'VRE']
Group 2: ['ACB', 'BID', 'CTG', 'HDB', 'LPB', 'MBB', 'SHB', 'SSB', 'STB', 'TCB', 'TPB', 'VCB', 'VPB']


In [ ]:
import os
import pandas as pd

groups_folder_path = os.path.join("processed_data/csv/inc_state")
group1 = []
group2 = []

# Lặp qua tất cả các file trong thư mục
for filename in os.listdir(groups_folder_path):
    if filename.endswith("_transformed.csv"):  # Chỉ xử lý các file có cấu trúc mong muốn
        file_path = os.path.join(groups_folder_path, filename)

        try:
            # Đọc chỉ tên cột của file CSV
            df = pd.read_csv(file_path, nrows=0)  # Đọc dòng header (tên cột)
            column_names = df.columns.astype(str).to_list()  # Chuyển danh sách cột thành string list

            stockcode = filename.replace("_transformed.csv", "")
            if any("Doanh thu bán hàng và cung cấp dịch vụ" in col for col in column_names):
                group1.append(stockcode)
            else:
                group2.append(stockcode)

        except Exception as e:
            print(f"Lỗi khi đọc file {filename}: {e}")

print("Group 1:", group1)
print("Group 2:", group2)

Group 1: ['BCM', 'FPT', 'GAS', 'GVR', 'HPG', 'MSN', 'MWG', 'PLX', 'SAB', 'VHM', 'VIC', 'VJC', 'VNM', 'VRE']
Group 2: ['ACB', 'BID', 'BVH', 'CTG', 'HDB', 'LPB', 'MBB', 'SHB', 'SSB', 'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VPB']


: 